In [26]:
from langchain_openai import ChatOpenAI
from crewai import Agent, Crew, Process, Task
from crewai.project import CrewBase, agent, crew, task
from crewai_tools import VisionTool
import os
from crewai import Agent, Task, Crew, Process
from dotenv import load_dotenv


In [27]:
load_dotenv()


openai_api_key = os.getenv('OPENAI_API_KEY')

In [48]:
os.environ["OPENAI_MODEL_NAME"] = "gpt-4o-mini"


llm = ChatOpenAI(
    model="gpt-3.5-turbo-16k",
    temperature=0.1,
    max_tokens=8000)

In [50]:
#Defining the URL
image_url = "https://encrypted-tbn3.gstatic.com/shopping?q=tbn:ANd9GcQQ8NVc0s_aQ-y97-jJlBpGQyHG8cbu1_Gr2i7htKhpE5giFJVy-ZSyaUcpXo7ExvF2H7aocQziYBqUGUx0EKNCVniAc0I37Rble0wFIu4HDah3hDrcfwryDKvS&usqp=CAE"

vision_tool = VisionTool()

image_text_extractor = Agent(
     role='Item Name & Description Extraction Expert',
     goal="Extract name of item along with their description present from images efficiently, you should get items name from %s"%image_url,
     backstory='You are an expert in extracting name of item present along with their description. Make sure you use the tools provided.',
     tools=[vision_tool], 
     allow_delegation=False, 
     verbose=True)


def text_extraction_task(agent):
        return Task(
            description = 
            """
            Extract name of item present along with their description from the provided image file. 
            Make sure that the item name & description is accurate and complete, 
            and ready for any further analysis or processing tasks. The image file provided may contain
            various products of Different brands, so it's crucial to capture all readable text. 
            """,
            agent = agent,
            expected_output = "A string containing NAME OF ITEM PRESENT and THEIR DESCRIPTION extracted from the image.",
            max_iter=1
        )



In [51]:
description_generator = Agent(
     role='Description Expert',
     goal='Generate a good description of the product(not any PERSON) from the item names & description extracted from the previous agent, make sure to highlight all its key features for displaying on a website',
     backstory='You are an expert in generating good descriptions for display on websites',
     llm=llm, 
     allow_delegation=False, 
     verbose=True)


def description_generator_task(agent):
        return Task(
            description = "Generate a good description of the product(not any PERSON) from the item names & description extracted by the previous agent, make sure to highlight all its key features for display on a website",
            agent = agent,
            expected_output = "A string containing a good description of the product.",
            max_iter=1)


In [52]:
title_generator = Agent(
     role='Item Title Expert',
     goal='Create a good title for the product (not any PERSON) from the item description generated by the previous agent in maximum of 3 words for display on a ecommerce website ',
     backstory='You are an expert in creating eye catching titles for displaying on websites',
     llm=llm, 
     allow_delegation=False, 
     verbose=True)


def title_generator_task(agent):
        return Task(
            description = "Add a good title for the product (not any PERSON) in maximum of 3 words for display on a ecommerce website. The Output should be Description of the Product generated from previous agent along with the Title",
            agent = agent,
            expected_output = "Output should be Description of the Product Generated from previous agent along with the Title",
            max_iter=1)

In [53]:
task1 = text_extraction_task(image_text_extractor)
task2 = description_generator_task(description_generator)
task3 = title_generator_task(title_generator)

#start crew
targetting_crew = Crew(
    agents=[image_text_extractor, description_generator, title_generator],
    tasks=[task1, task2, task3],
    verbose=True, 
  process=Process.sequential # Sequential process will have tasks executed one after the other and the outcome of the previous one is passed as extra content into this next.
  )

targetting_result = targetting_crew.kickoff()


# Agent: Item Name & Description Extraction Expert
## Task: 
            Extract name of item present along with their description from the provided image file. 
            Make sure that the item name & description is accurate and complete, 
            and ready for any further analysis or processing tasks. The image file provided may contain
            various products of Different brands, so it's crucial to capture all readable text. 
            


# Agent: Item Name & Description Extraction Expert
## Using tool: Vision Tool
## Tool Input: 
"{\"image_path_url\": \"https://encrypted-tbn3.gstatic.com/shopping?q=tbn:ANd9GcQQ8NVc0s_aQ-y97-jJlBpGQyHG8cbu1_Gr2i7htKhpE5giFJVy-ZSyaUcpXo7ExvF2H7aocQziYBqUGUx0EKNCVniAc0I37Rble0wFIu4HDah3hDrcfwryDKvS&usqp=CAE\"}"
## Tool Output: 
The image shows a rose gold wristwatch with a round face and a brown dial. The watch features simple hour markers, and the brand name "Daniel Wellington" is displayed on the dial. The bracelet is also made of rose